Recherche Sémantique de Livres.
On Utilise les embeddings OpenAI et ChromaDB pour rechercher des livres par similarité

In [1]:
import pandas as pd
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv

In [2]:
# Charger les variables d'environnement (clés API et tokens)
load_dotenv()

True

In [3]:
# Charger le CSV des livres nettoyés
df_livres = pd.read_csv("livres_nettoyes.csv")

print(f"\nNombre de livres chargés: {len(df_livres)}")
print(f"Colonnes disponibles: {list(df_livres.columns)}")

# Afficher un aperçu
print("\nAperçu des données:")
print(df_livres[["isbn13", "title", "categories"]].head())


Nombre de livres chargés : 5197
Colonnes disponibles : ['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count', 'titre_complet', 'description_avec_id']

Aperçu des données :
          isbn13                title                     categories
0  9780002005883               Gilead                        Fiction
1  9780002261982         Spider's Web  Detective and mystery stories
2  9780006178736       Rage of angels                        Fiction
3  9780006280897       The Four Loves                 Christian life
4  9780006280934  The Problem of Pain                 Christian life


In [4]:
# Vérifier que la colonne description_avec_id existe
if "description_avec_id" not in df_livres.columns:
    print("Colonne 'description_avec_id' non trouvée")
    print("Colonnes disponibles:", df_livres.columns.tolist())
else:
    print(f"\nColonne 'description_avec_id' vérifiée")
    print(f"Type de données: {type(df_livres['description_avec_id'].iloc[0])}")
    print(f"Première description (100 chars):")
    print(f"  {str(df_livres['description_avec_id'].iloc[0])[:100]}...")


Colonne 'description_avec_id' vérifiée
Type de données : <class 'str'>
Première description (100 chars) :
  9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gil...


In [6]:
#Preparer les descriptions pour la base vectorielle
# Sauvegarder chaque description sur une ligne (format attendu par TextLoader)
chemin_descriptions = "descriptions_etiquetees.txt"

with open(chemin_descriptions, "w", encoding="utf-8") as f:
    for idx, desc in enumerate(df_livres["description_avec_id"]):
        f.write(str(desc) + "\n")

print(f"Fichier créé: {chemin_descriptions}")
print(f"Nombre de lignes: {len(df_livres)}")

Fichier créé : descriptions_etiquetees.txt
Nombre de lignes: 5197


In [10]:
# On Verifie le fichier qu'on vient de creer
with open(chemin_descriptions, "r", encoding="utf-8") as f:
    lignes_echantillon = [f.readline().strip() for _ in range(3)]

print(f"Nombre de lignes testées:") #3

for i, ligne in enumerate(lignes_echantillon, 1):
    longueur = len(ligne)
    apercu = ligne[:80] + "..." if longueur > 80 else ligne
    print(f"  Ligne {i} ({longueur} caractères): {apercu}")

Nombre de lignes testées:
  Ligne 1 (1168 caractères) : 9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating fo...
  Ligne 2 (1214 caractères) : 9780002261982 A new 'Christie for Christmas' -- a full-length novel adapted from...
  Ligne 3 (373 caractères) : 9780006178736 A memorable, mesmerizing heroine Jennifer -- brilliant, beautiful,...


In [13]:
# Charger le fichier texte
try:
    documents_bruts = TextLoader(chemin_descriptions, encoding="utf-8").load()
    print(f"Document chargé")
except Exception as e:
    print(f"Erreur lors du chargement: {e}")

Document chargé


In [14]:
# Découper les documents (un document par ligne)
decoupeur = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separator="\n"
)

documents_decoupes = decoupeur.split_documents(documents_bruts)

Created a chunk of size 1168, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1088, which is longer than the specified 1000
Created a chunk of size 1189, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 2006, which is longer than the specified 1000
Created a chunk of size 1222, which is longer than the specified 1000
Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1057, which is longer than the specified 1000
Created a chunk of size 1270, which is longer than the specified 1000
Created a chunk of size 1635, which is longer than the specified 1000
Created a chunk of size 1128, which is longer than the specified 1000
Created a chunk of s

In [15]:
print(f"Nombre de documents après découpage: {len(documents_decoupes)}")
print(f"\nAperçu du premier document:")
print(f"  Contenu: {documents_decoupes[0].page_content[:100]}...")


Nombre de documents après découpage : 2925

Aperçu du premier document :
  Contenu : 9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gil...


In [16]:
# Créer la base vectorielle avec ChromaDB et les embeddings OpenAI
base_vectorielle = Chroma.from_documents(
    documents_decoupes,
    embedding=OpenAIEmbeddings()
)

In [17]:
# Ici, on extrait l'ISBN13 du contenu du résultat de recherche
# avec le parametre contenu_resultat (str): Contenu du document retourné par la recherche et
# finalement on retourne un int: ISBN13 extrait, ou None en cas d'erreur
def extraire_isbn_du_resultat(contenu_resultat):
    try:
        # L'ISBN est le premier "mot" (nombre) du contenu
        isbn_str = contenu_resultat.strip('"').split()[0]
        isbn = int(isbn_str)
        return isbn
    except (IndexError, ValueError) as e:
        print(f"Erreur lors de l'extraction de l'ISBN: {e}")
        return None

In [18]:
# Effectuer une recherche sémantique, par ex
requete_test = "Un thriller policier avec des enquêtes et du suspense"

print(f"Requête: {requete_test}\n")

Requête : Un thriller policier avec des enquêtes et du suspense



In [19]:
# Rechercher les 10 résultats les plus similaires
resultats_bruts = base_vectorielle.similarity_search(requete_test, k=10)

print(f"Nombre de résultats trouvés: {len(resultats_bruts)}")
print(f"\nPremier résultat:")
print(f"  {resultats_bruts[0].page_content[:100]}...")

Nombre de résultats trouvés : 10

Premier résultat :
  9780446605588 In this "intriguing" thriller (People), a New Orleans police officer kidnaps an attorn...


In [20]:
# Tester l'extraction de l'ISBN
isbn_premier = extraire_isbn_du_resultat(resultats_bruts[0].page_content)
print(f"\nISBN13 du premier résultat: {isbn_premier}")

if isbn_premier:
    livre = df_livres[df_livres["isbn13"] == isbn_premier]
    if not livre.empty:
        print("\nInformations du livre:")
        print(f"  Titre: {livre['title'].iloc[0]}")
        print(f"  Auteur(s): {livre['authors'].iloc[0]}")
        print(f"  Catégorie: {livre['categories'].iloc[0]}")


ISBN13 du premier résultat : 9780446605588

Informations du livre :
  Titre : Fat Tuesday
  Auteur(s) : Sandra Brown
  Catégorie : Fiction


In [21]:
# On cherche des livres basée sur la similarité sémantique.
# chercher_livres_par_semantique() effectue une recherche dans la base vectorielle et retourne
#les informations des livres les plus pertinents avec les parametres requete (str): Requête en langage naturel
#nb_resultats (int): Nombre de résultats souhaité et on retourne un dataFrame contenant les livres trouvés avec leurs infos
def chercher_livres_par_semantique(requete, nb_resultats=10):
    # Chercher plus de résultats pour avoir une meilleure sélection
    k_recherche = min(50, len(df_livres))

    # Effectuer la recherche sémantique
    resultats_bruts = base_vectorielle.similarity_search(requete, k=k_recherche)

    # Extraire les ISBNs de tous les résultats
    isbns_trouves = []

    for resultat in resultats_bruts:
        isbn = extraire_isbn_du_resultat(resultat.page_content)
        if isbn is not None:
            isbns_trouves.append(isbn)

    # Récupérer les informations des livres trouvés
    if isbns_trouves:
        livres_trouves = df_livres[df_livres["isbn13"].isin(isbns_trouves)].copy()

        # Retourner seulement le nombre demandé de résultats
        return livres_trouves.head(nb_resultats)
    else:
        return pd.DataFrame()

In [22]:
# Test de la fonction
requete_test_1 = "Un thriller policier avec des enquêtes et du suspense"

resultats_1 = chercher_livres_par_semantique(requete_test_1, nb_resultats=5)

print(f"Requête: {requete_test_1}")
print(f"\nNombre de résultats: {len(resultats_1)}")

if not resultats_1.empty:
    print("\nLivres trouvés:")
    print(resultats_1[["title", "authors", "categories"]].to_string(index=False))


Requête : Un thriller policier avec des enquêtes et du suspense

Nombre de résultats : 5

Livres trouvés :
                         title                         authors categories
               The Hidden Hand               Carroll John Daly    Fiction
                        Nausea Jean-Paul Sartre;Robert Baldick    Fiction
Raffles, the Amateur Cracksman          Ernest William Hornung    Fiction
           Voice of the Violin                Andrea Camilleri    Fiction
          The Bar on the Seine    Georges Simenon;David Watson    Fiction


In [23]:
# Pour plusieurs requetes ca donne
requetes_test = [
    "Un livre sur l'amour et les relations romantiques",
    "Des aventures spatiales avec des aliens et des voyages intergalactiques",
    "Un guide pratique pour apprendre la programmation et le développement web",
    "Des mystères et des secrets à découvrir",
    "Une histoire inspirante sur la persévérance et le courage"
]

resultats_tests = {}

for i, requete in enumerate(requetes_test, 1):
    print(f"\n{'-'*70}")
    print(f"Requête {i}: {requete}")
    print(f"{'-'*70}")

    resultats = chercher_livres_par_semantique(requete, nb_resultats=3)
    resultats_tests[requete] = resultats

    if not resultats.empty:
        print(f"Résultats ({len(resultats)} trouvés):\n")
        for idx, (_, livre) in enumerate(resultats.iterrows(), 1):
            print(f"{idx}. {livre['title']}")
            print(f"   Auteur(s): {livre['authors']}")
            print(f"   Catégorie: {livre['categories']}")
            print()
    else:
        print("Aucun résultat trouvé")


----------------------------------------------------------------------
Requête 1 : Un livre sur l'amour et les relations romantiques
----------------------------------------------------------------------
Résultats (3 trouvés) :

1. Identity
   Auteur(s) : Milan Kundera
   Catégorie : Fiction

2. The Infinite Plan
   Auteur(s) : Isabel Allende
   Catégorie : Fiction

3. Identity
   Auteur(s) : Milan Kundera
   Catégorie : Fiction


----------------------------------------------------------------------
Requête 2 : Des aventures spatiales avec des aliens et des voyages intergalactiques
----------------------------------------------------------------------
Résultats (3 trouvés) :

1. The Gap Into Madness
   Auteur(s) : Stephen R. Donaldson
   Catégorie : Hyland, Morn (Fictitious character)

2. The Far Side of Evil
   Auteur(s) : Sylvia Engdahl
   Catégorie : Juvenile Fiction

3. A Universal History of Iniquity
   Auteur(s) : Jorge Luis Borges;Andrew Hurley
   Catégorie : Fiction


-------

In [25]:
total_resultats = sum(len(r) for r in resultats_tests.values())
requetes_avec_resultats = sum(1 for r in resultats_tests.values() if len(r) > 0)

print(f"Nombre total de requêtes testées: {len(requetes_test)}")
print(f"Requêtes avec résultats: {requetes_avec_resultats}")
print(f"Total de résultats trouvés: {total_resultats}")

Nombre total de requêtes testées : 5
Requêtes avec résultats : 5
Total de résultats trouvés : 15


In [26]:
# On effectue une recherche interactive avec affichage détaillé avec requete (str): Requête de l'utilisateur comme parametre
# et on retourne un dataframe des résultats trouvés
def rechercher_interactive(requete):
    print(f"\n{'='*70}")
    print(f"Recherche: {requete}")
    print(f"{'='*70}\n")

    resultats = chercher_livres_par_semantique(requete, nb_resultats=10)

    if resultats.empty:
        print("Aucun résultat trouvé pour cette requête.")
        return resultats

    print(f"Résultats ({len(resultats)} livres):\n")

    for idx, (_, livre) in enumerate(resultats.iterrows(), 1):
        print(f"{idx}. {livre['title']}")
        print(f"   ISBN13: {livre['isbn13']}")
        print(f"   Auteur(s): {livre['authors']}")
        print(f"   Catégorie: {livre['categories']}")
        print(f"   Note moyenne: {livre['average_rating']:.2f}")
        print()

    return resultats

In [27]:
# Exemple 1
exemple_1 = "Un roman fantastique avec de la magie et des créatures mythologiques"
rechercher_interactive(exemple_1)


Recherche : Un roman fantastique avec de la magie et des créatures mythologiques

Résultats (10 livres) :

1. Fragile Things
   ISBN13 : 9780060515225
   Auteur(s) : Neil Gaiman
   Catégorie : Fiction
   Note moyenne : 4.01

2. Blood and Gold
   ISBN13 : 9780099271499
   Auteur(s) : Anne Rice
   Catégorie : Fiction
   Note moyenne : 3.89

3. Metamorphosis
   ISBN13 : 9780140447897
   Auteur(s) : Ovid;David Raeburn
   Catégorie : Fiction
   Note moyenne : 4.05

4. Mr Majeika and the School Trip
   ISBN13 : 9780141303352
   Auteur(s) : Humphrey Carpenter
   Catégorie : Juvenile Fiction
   Note moyenne : 3.81

5. A Universal History of Iniquity
   ISBN13 : 9780142437896
   Auteur(s) : Jorge Luis Borges;Andrew Hurley
   Catégorie : Fiction
   Note moyenne : 3.97

6. The Last Life
   ISBN13 : 9780156011655
   Auteur(s) : Claire Messud
   Catégorie : Fiction
   Note moyenne : 3.56

7. A Sentimental Education
   ISBN13 : 9780192836229
   Auteur(s) : Gustave Flaubert
   Catégorie : Fiction
  

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titre_complet,description_avec_id
123,9780060515225,0060515228,Fragile Things,Neil Gaiman,Fiction,http://books.google.com/books/content?id=ggLzJ...,A mysterious circus terrifies an audience for ...,2006.0,4.01,360.0,43210.0,Fragile Things: Short Fictions and Wonders,9780060515225 A mysterious circus terrifies an...
459,9780099271499,0099271494,Blood and Gold,Anne Rice,Fiction,http://books.google.com/books/content?id=kq8A8...,Here is the gorgeous and sinister story of Mar...,2002.0,3.89,752.0,27476.0,Blood and Gold: The Vampire Marius,9780099271499 Here is the gorgeous and siniste...
690,9780140447897,014044789X,Metamorphosis,Ovid;David Raeburn,Fiction,http://books.google.com/books/content?id=D3McX...,A new translation in hexameter verse of Ovid's...,2004.0,4.05,723.0,46550.0,Metamorphosis,9780140447897 A new translation in hexameter v...
771,9780141303352,0141303352,Mr Majeika and the School Trip,Humphrey Carpenter,Juvenile Fiction,http://books.google.com/books/content?id=YJ8eq...,"More amazing adventures with Mr Majeika, the e...",1999.0,3.81,89.0,45.0,Mr Majeika and the School Trip,9780141303352 More amazing adventures with Mr ...
832,9780142437896,0142437891,A Universal History of Iniquity,Jorge Luis Borges;Andrew Hurley,Fiction,http://books.google.com/books/content?id=DLRIP...,"In his writing, Borges always combined high se...",2004.0,3.97,128.0,2967.0,A Universal History of Iniquity,"9780142437896 In his writing, Borges always co..."
895,9780156011655,0156011654,The Last Life,Claire Messud,Fiction,http://books.google.com/books/content?id=NBiLm...,"In a story set between North Africa, France, a...",2000.0,3.56,400.0,1384.0,The Last Life,9780156011655 In a story set between North Afr...
973,9780192836229,0192836226,A Sentimental Education,Gustave Flaubert,Fiction,http://books.google.com/books/content?id=Dn7ix...,"Frederic Moreau, a moderately gifted young pro...",2000.0,3.83,464.0,104.0,A Sentimental Education: The Story of a Young Man,"9780192836229 Frederic Moreau, a moderately gi..."
1280,9780312873110,0312873115,The Land of Laughs,Jonathan Carroll,Fiction,http://books.google.com/books/content?id=OUIAm...,Have you ever loved a magical book above all o...,2001.0,3.90,256.0,4218.0,The Land of Laughs: A Novel,9780312873110 Have you ever loved a magical bo...
1441,9780345302359,0345302354,The Crucible of Time,John Brunner,Fiction,http://books.google.com/books/content?id=1T5dd...,Traces the development over milennia of a civi...,1984.0,3.71,416.0,24.0,The Crucible of Time,9780345302359 Traces the development over mile...
1452,9780345375216,0345375211,The Book of Lost Tales,John Ronald Reuel Tolkien,Fiction,http://books.google.com/books/content?id=Kwt_y...,The first form of the myths and legends in Tol...,1992.0,3.83,367.0,10413.0,The Book of Lost Tales,9780345375216 The first form of the myths and ...


In [28]:
# Exemple 2
exemple_2 = "Un essai sur l'histoire et la culture"
rechercher_interactive(exemple_2)


Recherche : Un essai sur l'histoire et la culture

Résultats (10 livres) :

1. The White Album
   ISBN13 : 9780006545866
   Auteur(s) : Joan Didion
   Catégorie : American essays
   Note moyenne : 4.17

2. Europe
   ISBN13 : 9780060974688
   Auteur(s) : Norman Davies
   Catégorie : History
   Note moyenne : 4.17

3. The Visual Arts
   ISBN13 : 9780131935075
   Auteur(s) : Hugh Honour;John Fleming
   Catégorie : Art
   Note moyenne : 3.82

4. East is East
   ISBN13 : 9780140131673
   Auteur(s) : T. Coraghessan Boyle
   Catégorie : Fiction
   Note moyenne : 3.64

5. Cod
   ISBN13 : 9780140275018
   Auteur(s) : Mark Kurlansky
   Catégorie : History
   Note moyenne : 3.90

6. Wanderlust
   ISBN13 : 9780140286014
   Auteur(s) : Rebecca Solnit
   Catégorie : Social Science
   Note moyenne : 3.92

7. Pleasing Myself
   ISBN13 : 9780140297973
   Auteur(s) : Frank Kermode
   Catégorie : American literature
   Note moyenne : 3.71

8. The Histories
   ISBN13 : 9780140449082
   Auteur(s) : Herodo

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titre_complet,description_avec_id
27,9780006545866,0006545866,The White Album,Joan Didion,American essays,http://books.google.com/books/content?id=qauOP...,This collection of essays recounts what took p...,1993.0,4.17,224.0,91.0,The White Album,9780006545866 This collection of essays recoun...
327,9780060974688,0060974680,Europe,Norman Davies,History,http://books.google.com/books/content?id=4StZD...,Here is a masterpiece of historical narrative ...,1998.0,4.17,1392.0,3242.0,Europe: A History,9780060974688 Here is a masterpiece of histori...
551,9780131935075,0131935070,The Visual Arts,Hugh Honour;John Fleming,Art,http://books.google.com/books/content?id=Z1p1M...,This new edition is an authoritative and provi...,2005.0,3.82,936.0,39.0,The Visual Arts: A History,9780131935075 This new edition is an authorita...
563,9780140131673,0140131671,East is East,T. Coraghessan Boyle,Fiction,http://books.google.com/books/content?id=5DHuA...,A young Japanese seaman jumps ship off the coa...,1991.0,3.64,384.0,2047.0,East is East,9780140131673 A young Japanese seaman jumps sh...
613,9780140275018,0140275010,Cod,Mark Kurlansky,History,http://books.google.com/books/content?id=6dKOl...,"A history of the fish that has led to wars, st...",1998.0,3.90,294.0,923.0,Cod: A Biography of the Fish that Changed the ...,9780140275018 A history of the fish that has l...
627,9780140286014,0140286012,Wanderlust,Rebecca Solnit,Social Science,http://books.google.com/books/content?id=_R2MD...,A cultural history of walking explores the anc...,2001.0,3.92,326.0,209.0,Wanderlust: A History of Walking,9780140286014 A cultural history of walking ex...
636,9780140297973,0140297979,Pleasing Myself,Frank Kermode,American literature,http://books.google.com/books/content?id=4rKpP...,Frank Kermode has often said that much of his ...,2002.0,3.71,278.0,12.0,Pleasing Myself: From Beowulf to Philip Roth,9780140297973 Frank Kermode has often said tha...
698,9780140449082,0140449086,The Histories,Herodotus,History,http://books.google.com/books/content?id=x-gAo...,One of the masterpieces of classical literatur...,2003.0,3.98,716.0,32923.0,The Histories,9780140449082 One of the masterpieces of class...
759,9780141185491,014118549X,Nausea,Jean-Paul Sartre;Robert Baldick,Fiction,http://books.google.com/books/content?id=EUM-P...,"In this novel, Antoine Roquentin, an introspec...",2000.0,3.92,253.0,1459.0,Nausea,"9780141185491 In this novel, Antoine Roquentin..."
864,9780143038313,0143038311,The Bar on the Seine,Georges Simenon;David Watson,Fiction,http://books.google.com/books/content?id=g1YkA...,Visiting a prisoner to inform him that his rep...,2007.0,3.69,160.0,375.0,The Bar on the Seine,9780143038313 Visiting a prisoner to inform hi...
